In [124]:
import requests
from bs4 import BeautifulSoup
import pandas as pd



In [93]:
class Review:
    
    def __init__(self, page_num):
        self.page_num = page_num
        self.url = f'https://onebite.app/reviews/dave?page={page_num}&minScore=0&maxScore=10'
        self.page = requests.get(self.url)
        self.soup = BeautifulSoup(self.page.text, 'html.parser')
        self.review_divs = []
        
    def get_divs(self, page_num):
        all_divs = self.soup.find_all('div')
        for div in all_divs:
            if 'feedItem' in f'{div.get("class")}':
                self.review_divs.append(div)
        
    def get_Title(self):
        titles = []
        for div in self.review_divs:
            title = div.find('h2').getText()
            titles.append(title)
        
        return titles
    
    def get_Rating(self):
        ratings = []
        for div in self.review_divs:
            rating = div.find('p', {'class': 'jsx-407081529 rating__score'}).getText()
            ratings.append(rating)
            
        return ratings
        
    def get_Location(self):
        locations = []
        for div in self.review_divs:
            location = div.find('p', {'class': 'jsx-2655995184 reviewCard__location'}).getText()
            locations.append(location)
            
        return locations

In [107]:
page_one = Review(1)

In [108]:
divs = page_one.get_divs(page_one.page_num)
titles = page_one.get_Title()
ratings = page_one.get_Rating()
locations = page_one.get_Location()

pizzaDict = {}

for num in range(len(titles)):
    title = titles[num]
    rating = ratings[num]
    location = locations[num]
    
    pizzaDict[title] = {'Rating': rating, 'Location': location}
    

In [159]:
official_pizza = []

for num in range(1,53):
    reviews = Review(num)
    get_divs = reviews.get_divs(reviews.page_num)
    get_titles = reviews.get_Title()
    get_ratings = reviews.get_Rating()
    get_locations = reviews.get_Location()
    
    for nums in range(len(get_titles)):
        location = get_locations[nums]
        comma = location.index(',')
        city = location[:comma]
        state = location[comma + 2:]
        if city == 'London':
            state = 'UK'
        if city == 'Griante' or city == 'Tremezzo' or city == 'Como' or city == 'Bellagio' or city == 'Milan' or city == 'Venezia':
            state = 'Italy'
        pizzareview = [get_titles[nums], get_ratings[nums], city, state]
        official_pizza.append(pizzareview)

In [160]:
pizza_df =  pd.DataFrame({"Restaurant Name":[i[0] for i in official_pizza],
                         "Rating":[i[1] for i in official_pizza],
                            'City':[i[2] for i in official_pizza],
                             'State': [i[3] for i in official_pizza]})

In [161]:
pizza_df.to_excel('PizzaReviewTime3.xlsx')